In [1]:
import numpy as np
# import tensorflow
import cv2
import time
import os
import mediapipe as mp
from numpy import expand_dims , where ,argmax
# from tensorflow.keras.preprocessing.image import  img_to_array


os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

In [2]:
def check_cheating(ROI):
    ROI.flags.writeable = False
    posingEstimation = face_mesh.process(ROI)
    ROI.flags.writeable = True
#////////////////////////////////////////////////////////////////////////////////////////
    rows, cols, _ =ROI.shape
    face_3d = []
    face_2d = []
#///////////////////////////////////////////////////////////////////////////////////
    if posingEstimation.multi_face_landmarks:
        for facelms in posingEstimation.multi_face_landmarks:
#//////////////////////////////////////////////////////////////////////////////
            for id ,lm in enumerate(facelms.landmark):
                a,b=int(lm.x*cols),int(lm.y*rows)
                if id in [1] :
                    #cv2.circle(frame,(a,b),10,(0,0,255),-1)

                    nose_2d=(a , b)
                    nose_3d=(a , b , lm.z * 3000)

                x , y = int(a), int(b)
                face_2d.append([x,y])
                face_3d.append([x,y,lm.z])
            face_2d = np.array(face_2d, dtype=np.float64)
            face_3d = np.array(face_3d, dtype=np.float64)
#////////////////////////////////////////////////////////////////////////////////////
            focal_len = 1* c
            cam_matrix = np.array([ [focal_len , 0 , r/2],
                                    [0 , focal_len , c/2],
                                    [0,0,1]])
            dist_matrix= np.zeros((4,1), dtype=np.float64)
#//////////////////////////////////////////////////////////////////////////////////////////////
            success , rot_vec , trans_vec =cv2.solvePnP(face_3d, face_2d , cam_matrix , dist_matrix)
            rmat , jac= cv2.Rodrigues(rot_vec)
            angles , mtxR , mtxQ , Qx , Qy , Qz= cv2.RQDecomp3x3(rmat)
            x=angles[0] * 360
            y=angles[1] * 360
            z=angles[2] * 360

#//////////////////////////////////////////////////////////////////////////////////////////
           
            if x < -40:     # head tilted to the left
                text = "Cheating"
            elif x > 15:     # head tilted to the right
                text = "Cheating"
            elif y < -40:      # looking down
                text = "Cheating"
            elif y > 40:     # looking up
                text = "Normal"
            else:            
                text = "Normal"

        return text
    

In [3]:
time_id = {}
time_temp = {}
student_id={}

In [4]:
def process_track(text, track_id):
    if text == "Cheating":
        if track_id not in time_temp:
            # If the track_id doesn't exist, add it with the current time in milliseconds
            time_temp[track_id] = [int(time.time()) , True]

    elif text == "Normal":
        # Calculate the difference between the current time and the previous cheating time
        if track_id in time_temp and time_temp[track_id][1]:
            current_time = int(time.time())
            time_difference = current_time - time_temp[track_id][0]
            # Update the time for the track_id
            if track_id in time_id:
                time_id[track_id] += time_difference
            else:
                time_id[track_id] = time_difference
            time_temp.pop(track_id)
    

In [5]:
def preprocess_image(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,dsize=(224,224),interpolation=cv2.INTER_CUBIC)
    img = img_to_array(img)
    img = img/255.0
    img = expand_dims(img, axis=0)
    return img

In [6]:
model = YOLO(r"C:\\Users\\DELL\\Downloads\\YoloV8 Face.pt")
names = model.model.names

In [7]:
mobile_detection_model = YOLO(r"c:\\Users\\dell\\OneDrive\\Desktop\\Graduation project\\best best (mobile) .pt")
mobile_detection_names = mobile_detection_model.model.names

In [8]:
mp_drawing = mp.solutions.drawing_utils
mpFacemesh = mp.solutions.face_mesh
mp_drawing_styles = mp.solutions.drawing_styles
face_mesh = mpFacemesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [9]:
mobile=False

# face recognition

In [10]:
from tensorflow.keras.layers import ZeroPadding2D, Convolution2D, \
MaxPooling2D, Dropout, Flatten, Activation
from tensorflow.keras.models import Sequential ,load_model
from tensorflow.keras.models import Model
def vgg_face():
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))
    return model

In [11]:
model_vgg = vgg_face()
model_vgg = Model(inputs=model_vgg.layers[0].input, outputs=model_vgg.layers[-2].output)
model_vgg.load_weights('c:\\Users\\dell\\OneDrive\\Desktop\\Graduation project\\vgg_face_weights.h5')

d:\Anconda\envs\mediapipe_env\lib\site-packages\keras\src\layers\reshaping\zero_padding2d.py:72: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
from tensorflow.keras.preprocessing.image import  img_to_array


# Preprocessing of image 
def preprocess_image(img):
    img = img_to_array(img)
    img = img/255.0
    img = expand_dims(img, axis=0)
    return img

# to identify the student
def Face_Recognition(roi):
    roi = cv2.cvtColor(roi,cv2.COLOR_BGR2RGB)
    roi=cv2.resize(roi,dsize=(224,224),interpolation=cv2.INTER_CUBIC)
    roi=preprocess_image(roi)
    embedding_vector = model_vgg.predict(roi)[0]
    return embedding_vector


def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

In [13]:
import pandas as pd

# Create an empty DataFrame with 2623 columns
columns = [f"Column_{i}" for i in range(1, 2624)]
points_of_face = pd.DataFrame(columns=columns)

In [14]:
student_id_temp={}
student_id={}

In [15]:
# Load video
cap = cv2.VideoCapture(0) # "video.mp4"
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set video codec and frame rate
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('outputzz11.avi', fourcc, 30, (width, height))  # Adjust frame rate (30 fps in this example)

session_start=int(time.time())

while cap.isOpened():
    success, frame = cap.read()
    if not success or (cv2.waitKey(1) & 0xFF == ord("q")):
        break
    r, c, _ = frame.shape
    annotator = Annotator(frame, line_width=2)

    face_recognized=set() # reset the set which contains faces that appear with each frame
    removed_ids=[]
   ########################################################################################################################
   # Mobile_detection_Tracking

    result1=mobile_detection_model(frame)

    for result in result1[0].boxes.cpu().numpy():
            x1,y1,x2,y2=map(int,result.xyxy[0])
            cls=names[int(result.cls[0])]
            conf=result.conf[0].round(2)
           
            if conf >0.90 :
                mobile=True
   ################################################################################################################################
   # Cheating_detection_tracking

    results = model.track(frame, iou=0.6, show=False, tracker="bytetrack.yaml", persist=True, verbose=False)

    if results[0].boxes.id is not None:
        track_ids = results[0].boxes.id.int().cpu().tolist()
        cls = names[results[0].boxes.cls.int().cpu().tolist()[0]]
        boxes = results[0].boxes.xyxy.cpu()
        conf = results[0].boxes.conf.tolist() 
        for box, track_id, cof, cc in zip(boxes, track_ids, conf, cls):
            x1, y1, x2, y2 = box.int().tolist()   
            ROI = frame[y1:y2, x1:x2]
            if ROI.size != 0 :
                ROI = cv2.cvtColor(ROI,cv2.COLOR_BGR2RGB)
                text = check_cheating(ROI)

                emded=Face_Recognition(ROI)
                flag=False # define the flag to determine if the person appears for the first time or not
                id=track_id 
                for i in range(points_of_face.shape[0]):
                    sim=findCosineSimilarity(emded,points_of_face.iloc[i,:-1])
                    if sim > 0.2 :
                        id=points_of_face.iloc[i,-1] # if person appear in the past -----> doesn't take new id
                        flag=True
                    
                        
                        
                face_recognized.add(id)   
                if flag == False:
                    x=[] # list to collect vector of face_recognition (emded) and id 
                    for value in emded:
                        x.append(value) 
                    x.append(track_id)
                    
                    points_of_face.loc[len(points_of_face)] = (x)   # add new person to points_of_face dataframe
################################################################################################################################
                if id not in time_id:
                    time_id[id] = 0
                process_track(text, id)
################################################################################
                # calculate time for each person appear

                for i in face_recognized:
                    if i in student_id_temp.keys(): # if the person appears in the  frame (appears in face_recognized list) and he existed in previous frames ---> then he continues ---> then update last_seen_time only
                            student_id_temp[i]['last_seen_time']=int(time.time())
                    else: # he appears in the  frame (appears in face_recognized list) and he wasn't exist in previous frames -----> then he appears for the first time
                        student_id_temp[i]={
                            'first_seen_time': int(time.time()),
                            'last_seen_time': int(time.time())
                        }   


                for j in student_id_temp.keys():  
                    if j not in face_recognized:# if the person exists in (student_id_temp) and not exists in (face_recognized) ----> then he is leave session ----> then I can calculate the duration that he appeared in it
                        if j not in student_id:
                            student_id[j] = (student_id_temp[j]['last_seen_time']-student_id_temp[j]['first_seen_time'])
                        else:
                            student_id[j] += (student_id_temp[j]['last_seen_time']-student_id_temp[j]['first_seen_time'])   

                        removed_ids.append(j) 


                for id in removed_ids: # remove the person that leaves from (student_id_temp) ------> bec if he appears again calculate time correctly by resetting (first_seen_time)
                    student_id_temp.pop(id,None)      
               
            
################################################################################################################################################
    # Write frame to output video
    out.write(frame)
    # Display frame
    cv2.imshow('img', frame)


# Release video capture and writer
cap.release()
out.release()
cv2.destroyAllWindows()

session_end=int(time.time())


0: 480x640 (no detections), 253.4ms
Speed: 4.3ms preprocess, 253.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 752ms/step

0: 480x640 (no detections), 293.1ms
Speed: 6.4ms preprocess, 293.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step

0: 480x640 (no detections), 243.7ms
Speed: 3.3ms preprocess, 243.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step

0: 480x640 (no detections), 242.8ms
Speed: 2.0ms preprocess, 242.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step

0: 480x640 (no detections), 234.5ms
Speed: 2.0ms preprocess, 234.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 406ms/step

0: 480x640 (no detections), 217.2ms
Speed: 2.0ms preprocess, 217.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 

# preprocessing

In [16]:
# calculate the time for people who exist until the end
for j in student_id_temp.keys():
    if j not in student_id:
        student_id[j] = (student_id_temp[j]['last_seen_time']-student_id_temp[j]['first_seen_time'])
    else:
        student_id[j] += (student_id_temp[j]['last_seen_time']-student_id_temp[j]['first_seen_time']) 

In [17]:
student_id

{1: 9}

In [18]:
time_id # contain -----> {id of student , time who is take turns his face}

{1: 5}

In [19]:
student_duration=0
for key,value in student_id.items():
    if value >= student_duration:
        student_duration=value
        id_of_real_student=key

In [20]:
total_time_of_session = session_end - session_start

# Determine if the student is a cheater or polite

In [21]:
total_score=100

In [22]:
result='polite'

if (len(student_id)>1 ): # if more than one person is appearing in the frame and it takes a time of more than 3 seconds
    result="Cheater"
    total_score-=20
if time_id[id_of_real_student] > 15:  # if a student turns around and takes time more than 15 seconds 
    result="Cheater"
    total_score-=10
if mobile: # if the student holds the mobile phone
    result="Cheater"
    total_score-=30


print(result)

polite


In [23]:
if total_time_of_session - student_duration > 10: # If the student closes (covers) the camera or leavea the exam
    total_score=0                                       # Then he cheated all the session ------> lost all the score

In [24]:
percentage_of_cheating=(100-total_score)/100

In [25]:
print(result,percentage_of_cheating)

polite 0.0


In [26]:
percentage_of_cheating

0.0

# Report

In [27]:
# if the student holds the mobile phone or not
mobile

False

In [28]:
# how many time a student turns around 
time_id[id_of_real_student]

5

In [29]:
# how many time the student doesn't attend the session or try to covers the camera
(total_time_of_session - student_duration)

4

In [30]:
# how many another people exist during the session and how many time they attend
count_of_another_people=0
time_of_another_people=0


for i,j in student_id.items():
    if( i != id_of_real_student):
       count_of_another_people +=1
       time_of_another_people += j


print(count_of_another_people)
print(time_of_another_people)

0
0


In [31]:
# try to open another tab or take a screenshots